# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [ ]:
! pip install evaluate
! pip install scikit-learn
! pip install -q "datasets==2.15.0"

In [ ]:
#Load and preprocess a dataset
# Load the sms_spam dataset
# See: https://huggingface.co/datasets/sms_spam

from datasets import load_dataset

# The sms_spam dataset only has a train split, so we use the train_test_split method to split it into train and test
dataset = load_dataset("sms_spam", split="train").train_test_split(
    test_size=0.2, shuffle=True, seed=23
)

splits = ["train", "test"]

# View the dataset characteristics
dataset["train"]

In [ ]:
# Inspect the first example. Do you think this is spam or not?
dataset["train"][0]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Let's use a lambda function to tokenize all the examples
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["sms"], truncation=True), batched=True
    )

# Inspect the available columns in the dataset
tokenized_dataset["train"]


In [ ]:
# Load a pretrained HF(huggingface) foundation modela
# In this case we are doing a full fine tuning, so we will want to unfreeze all parameters.

from transformers import AutoModelForSequenceClassification

full_model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "not spam", 1: "spam"},
    label2id={"not spam": 0, "spam": 1},
)

# Unfreeze all the model parameters.
# Hint: Check the documentation at https://huggingface.co/transformers/v4.2.2/training.html
for param in full_model.parameters():
    param.requires_grad = True

In [ ]:
print(full_model)

In [ ]:
# Let's train/full fine-tune it!
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=full_model,
    args=TrainingArguments(
        output_dir="./data/spam_not_spam",
        # Set the learning rate
        learning_rate=2e-5,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()


In [ ]:
# Evaluate the full fine-tuned model
trainer.evaluate()

In [ ]:
# Make a dataframe with the predictions and the text and the labels
import pandas as pd

items_for_manual_review = tokenized_dataset["test"].select(
    [0, 1, 22, 31, 43, 292, 448, 487]
)

results = trainer.predict(items_for_manual_review)
df = pd.DataFrame(
    {
        "sms": [item["sms"] for item in items_for_manual_review],
        "predictions": results.predictions.argmax(axis=1),
        "labels": results.label_ids,
    }
)
# Show all the cell
pd.set_option("display.max_colwidth", None)
df


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [ ]:
# Create a PEFT model

from peft import get_peft_config, get_peft_model, LoraConfig, TaskType


In [ ]:

#model_name_or_path = "bigscience/mt0-large"
#tokenizer_name_or_path = "bigscience/mt0-large"

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, 
    inference_mode=False, 
    r=8, 
    target_modules=["q_lin", "v_lin"],
    lora_alpha=32, 
    lora_dropout=0.1
)

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased"
)

# creating a PEFT model
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()


In [ ]:
# Training/fine-tuning the PEFT model
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=peft_model,
    args=TrainingArguments(
        output_dir="./data/spam_not_spam",
        # Set the learning rate
        learning_rate=2e-5,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        
        num_train_epochs=1,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
# Make a dataframe with the predictions and the text and the labels
import pandas as pd

items_for_manual_review = tokenized_dataset["test"].select(
    [0, 1, 22, 31, 43, 292, 448, 487]
)

results = trainer.predict(items_for_manual_review)
df = pd.DataFrame(
    {
        "sms": [item["sms"] for item in items_for_manual_review],
        "predictions": results.predictions.argmax(axis=1),
        "labels": results.label_ids,
    }
)
# Show all the cell
pd.set_option("display.max_colwidth", None)
df

In [ ]:
#Saving the trained model
peft_model.save_pretrained("./peft_model")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [ ]:
# Load the saved PEFT model
from peft import PeftModel, PeftConfig, AutoPeftModelForSequenceClassification
from transformers import AutoModelForSequenceClassification

In [ ]:
peft_model_id = "./peft_model"
config = PeftConfig.from_pretrained(peft_model_id)

In [ ]:
model = AutoPeftModelForSequenceClassification.from_pretrained(peft_model_id)
model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

In [ ]:
################ test stuff #######################
#https://huggingface.co/docs/transformers/main/en/model_doc/distilbert#transformers.DistilBertForSequenceClassification
x = tokenized_dataset["test"].select(
    [0, 1, 22, 31, 43, 292, 448, 487]
)

print(x)

for i in x:
    print(i['input_ids'])
    print(i)
    break

In [ ]:
# Make a dataframe with the predictions and the text and the labels
import pandas as pd
import torch

items_for_manual_review = tokenized_dataset["test"].select(
    [0, 1, 22, 31, 43, 292, 448, 487]
)

predictions = []
for i in items_for_manual_review:
    input_tokens = tokenizer(i['sms'], return_tensors="pt")
    #print(input_tokens)
    with torch.no_grad():
        logits = model(**input_tokens).logits
        predicted_class_id = logits.argmax().item()
        predictions.append(predicted_class_id)

In [ ]:
print(predictions)

In [ ]:
df = pd.DataFrame(
    {
        "sms": [item["sms"] for item in items_for_manual_review],
        #"predictions": results.predictions.argmax(axis=1),
        "predictions": predictions,
        #"labels": results.label_ids,
        "labels": [item["label"] for item in items_for_manual_review],
    }
)
# Show all the cell
pd.set_option("display.max_colwidth", None)
df

In [ ]:
# Evaluate the fine-tuned model
labels = [item["label"] for item in items_for_manual_review]

In [ ]:
import evaluate

In [ ]:
accuracy_metric = evaluate.load("accuracy")

In [ ]:
results = accuracy_metric.compute(references=labels, predictions=predictions)
print(results)
